# Federated PyTorch MNIST Tutorial

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision ..

Processing /home/itrushkin/repos/openfl-fork


  Created wheel for openfl: filename=openfl-1.0.1-py3-none-any.whl size=4347476 sha256=74ad0681e425345b8f25eaaf23b3ec453f0d3765306d2f6ececae2989169dbd2
  Stored in directory: /home/itrushkin/.cache/pip/wheels/ed/ad/82/f26baafab4be32d052fb76a6f56c9a76ce6903cc3eed7bb394
Successfully built openfl
  Attempting uninstall: openfl
    Found existing installation: openfl 1.0.1
    Uninstalling openfl-1.0.1:
      Successfully uninstalled openfl-1.0.1
You should consider upgrading via the '/home/itrushkin/.virtualenvs/openfl_research/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
├── agg_to_col_two_signed_cert.zip
├── data
│   └── MNIST
│       ├── raw
│       └── processed
├── cert
│   ├── ca
│   │   ├── signing-ca.crt
│   │   ├── root-ca.crt
│   │   ├── signing-ca
│   │   ├── signing-ca.csr
│   │   └── root-ca
│   ├── server
│   │   ├── agg_nnlicv611.inn.intel.com.csr
│   │   ├── agg_nnlicv611.inn.intel.com.key
│   │   └── agg_nnlicv611.inn.intel.com.crt
│   ├── client
│   │   ├── col_two.crt
│   │   ├── col_one.crt
│   │   ├── col_one.key
│   │   └── col_

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=torch.argmax(target, dim=1))

In [6]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [7]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [8]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 30000
Collaborator one's validation data size: 5000

Collaborator two's training data size: 30000
Collaborator two's validation data size: 5000



We can see the current plan values by running the `fx.get_plan()` function

In [9]:
 #Get the current values of the plan. Each of these can be overridden
import json
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.template": "openfl.component.Collaborator",
    "data_loader.settings.batch_size": 2

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [12]:
#Run experiment, return trained FederatedModel
from openfl.component.aggregation_functions import AggregationFunctionInterface

class ClippedAveraging(AggregationFunctionInterface):
    def clip(self, tensor):
        # clip could be any number of clipping functions
        return tensor * .3
    def __call__(self, tensors: np.ndarray, **kwargs) -> np.ndarray:
        clipped_tensors = []
        for tensor in tensors:
            previous_tensor_value = kwargs['previous_tensor_value'] or tensor
            # contents of "history" would be well documented
            delta = tensor - previous_tensor_value
            clipped_tensor = previous_tensor_value + self.clip(delta) 
            clipped_tensors.append(clipped_tensor)

        return np.average(clipped_tensors, weights=kwargs['weights'], axis=0)
    
final_fl_model = fx.run_experiment(collaborators,
                                   {
                                       'aggregator.settings.rounds_to_train':5,
                                       'tasks.train.aggregation_type': ClippedAveraging()
                                   })

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:154: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


In [11]:
from openfl.databases import TensorDB

TensorDB.__attrs__

AttributeError: type object 'TensorDB' has no attribute '__attrs__'

In [ ]:
#Save final model
final_fl_model.save_native('final_pytorch_model')